In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import analysis
import db
import fetcher
from recordings import Recording
from trim_recordings import detect_utterances

import IPython.display
from IPython.display import display
import librosa
import librosa.display
import librosa.feature
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import numpy as np
import pydub
import scipy.ndimage
from tqdm import tqdm

import hashlib
import io
import multiprocessing

plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
session = db.create_session('master.db')
recordings_fetcher = fetcher.Fetcher('recordings', pool_size=8)

Load recordings from the database and filter them according to some selection criteria:
right species, contains the data we need, good quality, not too short and not too long.

Then get them from the cache or download them if necessary.

In [ ]:
def md5(string):
    m = hashlib.md5()
    m.update(string.encode('utf-8'))
    return m.digest()

recordings = [
    r for r in session.query(Recording).filter(
        #Recording.genus == 'Turdus', Recording.species == 'merula', # Merel
        #Recording.genus == 'Passer', Recording.species == 'domesticus', # Huismus
        Recording.genus == 'Parus', Recording.species == 'major', # Koolmees
        #Recording.genus == 'Acrocephalus', Recording.species == 'palustris', # Bosrietzanger
        #Recording.genus == 'Botaurus', Recording.species == 'stellaris', # Roerdomp
    )
    if r.url and r.audio_url and not r.background_species and r.quality == 'A' and 10 <= r.length_seconds <= 120
]
recordings.sort(key=lambda recording: md5(recording.recording_id))
print(f'Found {len(recordings)} candidate recordings')

recordings = recordings[:12]
#recordings = recordings[12:24]
recordings = {r.recording_id: r for r in recordings}

def load_recording(recording):
    data = recordings_fetcher.fetch_cached(recording.audio_url)
    sound = analysis.load_sound(io.BytesIO(data))
    return (recording.recording_id, sound)

pool = multiprocessing.pool.Pool(8)
sounds = dict(tqdm(pool.imap(load_recording, recordings.values(), 1), total=len(recordings)))
pool.close()

Analyze each recording and plot the intermediate and final results.

In [ ]:
for recording_id, sound in sounds.items():
    # Perform analysis.
    mel_spectrogram = analysis.mel_spectrogram(sound)
    noise_profile = analysis.noise_profile(mel_spectrogram)
    noise_volume_db = analysis.noise_volume_db(noise_profile)
    filtered_volume_db = analysis.filtered_volume_db(mel_spectrogram, noise_profile)
    vocalizations = analysis.filter_vocalizations(
        analysis.merge_vocalizations(
            analysis.vocalizations(filtered_volume_db, noise_volume_db)))
    
    # Create figure.
    fig, ax = plt.subplots(1, 2, figsize=(10, 3), gridspec_kw={'width_ratios': [8, 1]}, sharey='all')
    
    # Display spectrogram on the left.
    img = librosa.display.specshow(librosa.power_to_db(mel_spectrogram),
                                   x_axis='time', y_axis='mel',
                                   cmap='magma', vmin=-80, vmax=0,
                                   hop_length=hop_length, sr=analysis.SAMPLE_RATE, ax=ax[0])
    ax[0].set_title(recording_id)
    ax[0].title.set_url('https:' + recordings[recording_id].url)
    
    # Display vocalizations as transparent green rectangles on top of the spectrogram.
    for (start, end) in vocalizations:
        ax[0].add_patch(
            Rectangle((start, 0), end - start, analysis.SAMPLE_RATE / 2,
                      edgecolor='none', facecolor='#00ff0050'))
    
    # Plot volume of noise-filtered signal on top of spectrogram.
    ax_right = ax[0].twinx()
    xs = (np.arange(0, mel_spectrogram.shape[1]) + 0.5) * analysis.FFT_HOP_LENGTH / analysis.SAMPLE_RATE
    ax_right.plot(xs, filtered_volume_db,
                  linewidth=1.0)
    ax_right.set_ylim(-80, 0)
    
    # Add lines at our vocalization thresholds.
    ax_right.axhline(noise_volume_db, color='red')
    ax_right.axhline(noise_volume_db + analysis.VOCALIZATION_TRIGGER_THRESHOLD_DB, color='yellow')
    ax_right.axhline(noise_volume_db + analysis.VOCALIZATION_KEEP_THRESHOLD_DB, color='green')
    
    # Display noise profile on the right.
    img = librosa.display.specshow(librosa.power_to_db(noise_profile),
                                   y_axis='mel', ax=ax[1],
                                   cmap='magma', vmin=-80, vmax=0)
    ax[1].set_xticks([])
    ax[1].set_ylabel(None)
    ax[1].set_title(f'Noise: {noise_volume_db:.1f} dB')
    fig.colorbar(img, ax=ax[1])
    
    plt.tight_layout()
    plt.show()
    
    display(IPython.display.Audio(sound, rate=analysis.SAMPLE_RATE))

In [ ]:
# Just a large empty cell to end with, to prevent scrolling when re-evaluating the output of the previous cell.





















































